In [2]:
import tensorflow as tf
print("✅ TensorFlow version:", tf.__version__)
print("✅ GPU available:", tf.config.list_physical_devices('GPU'))


✅ TensorFlow version: 2.19.0
✅ GPU available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [1]:
# 📦 Full Kaggle Dataset Setup Script

from google.colab import files
files.upload()   # Upload kaggle.json



Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"bikrammaity","key":"4d82303703c11ec6fd5dabe20c67ccee"}'}

In [2]:
# ✅ Create a directory for Kaggle credentials
!mkdir -p ~/.kaggle

# ✅ Move your Kaggle API key (make sure you've uploaded kaggle.json to Colab)
!mv kaggle.json ~/.kaggle/

# ✅ Set secure permissions for the key file
!chmod 600 ~/.kaggle/kaggle.json

# ✅ Download the Melanoma Skin Cancer dataset from Kaggle
!kaggle datasets download -d hasnainjaved/melanoma-skin-cancer-dataset-of-10000-images

# ✅ Unzip it into a 'data' directory
!unzip -q melanoma-skin-cancer-dataset-of-10000-images.zip -d ./data

# ✅ Remove the zip file to save space
!rm melanoma-skin-cancer-dataset-of-10000-images.zip

# ✅ List the contents of the extracted dataset
!ls ./data


Dataset URL: https://www.kaggle.com/datasets/hasnainjaved/melanoma-skin-cancer-dataset-of-10000-images
License(s): CC0-1.0
  0% 0.00/98.7M [00:00<?, ?B/s]
100% 98.7M/98.7M [00:00<00:00, 1.53GB/s]
melanoma_cancer_dataset


In [3]:
import os
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense,Dropout,Rescaling,RandomFlip,RandomRotation,RandomZoom,BatchNormalization,GlobalAveragePooling2D,RandomContrast,RandomBrightness
from tensorflow.keras import layers

In [4]:
data_dir="/content/data/melanoma_cancer_dataset"
train_dir=os.path.join(data_dir,"train")
test_dir=os.path.join(data_dir,"test")


In [4]:
train_ds=keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=(150,150),
    batch_size=32,
    shuffle=True,
    color_mode="rgb",
    label_mode="binary",
    labels="inferred"
)

test_ds=keras.utils.image_dataset_from_directory(
    test_dir,
    image_size=(150,150),
    batch_size=32,
    shuffle=False,
    color_mode="rgb",
    label_mode="binary",
    labels="inferred"
)

Found 9605 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.


In [5]:
class_names=train_ds.class_names
print(class_names)

['benign', 'malignant']


In [6]:
from tensorflow.data import AUTOTUNE
train_ds=train_ds.cache().shuffle(5000).prefetch(buffer_size=AUTOTUNE)
test_ds=test_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [7]:
#perform Data Augmentation
data_augmentation = keras.Sequential(
    [
        RandomFlip("horizontal"),
        RandomRotation(0.1),
        RandomZoom(0.2),
    ]
)

In [8]:
#CReate Deep Learning Model
model=Sequential()
model.add(layers.Input(shape=(150, 150, 3)))
model.add(data_augmentation)
model.add(Rescaling(1./255))
model.add(Conv2D(256,(3,3),padding='same',activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(256,(3,3),padding='same',activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(128,(3,3),padding='same',activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(256,(3,3),padding='same',activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))

In [9]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential (Sequential)         │ (None, 150, 150, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling (Rescaling)           │ (None, 150, 150, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 150, 150, 256)  │         7,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 75, 75, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 75, 75, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 37, 37, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 37, 37, 128)    │       295,040 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 18, 18, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 18, 18, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 9, 9, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 20736)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     2,654,336 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,858,433 (14.72 MB)

 Trainable params: 3,858,433 (14.72 MB)

 Non-trainable params: 0 (0.00 B)

In [10]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)


In [11]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [12]:
cnn=model.fit(
    train_ds,
    validation_data=test_ds,
    epochs=80,
    callbacks=[early_stopping],
    verbose=1,
)

Epoch 1/80
301/301 ━━━━━━━━━━━━━━━━━━━━ 61s 141ms/step - accuracy: 0.6087 - loss: 0.6438 - val_accuracy: 0.8490 - val_loss: 0.4472
Epoch 2/80
301/301 ━━━━━━━━━━━━━━━━━━━━ 40s 133ms/step - accuracy: 0.7966 - loss: 0.4487 - val_accuracy: 0.8800 - val_loss: 0.3024
Epoch 3/80
301/301 ━━━━━━━━━━━━━━━━━━━━ 40s 134ms/step - accuracy: 0.8416 - loss: 0.3694 - val_accuracy: 0.8590 - val_loss: 0.3503
Epoch 4/80
301/301 ━━━━━━━━━━━━━━━━━━━━ 40s 133ms/step - accuracy: 0.8582 - loss: 0.3463 - val_accuracy: 0.8930 - val_loss: 0.2554
Epoch 5/80
301/301 ━━━━━━━━━━━━━━━━━━━━ 40s 134ms/step - accuracy: 0.8649 - loss: 0.3298 - val_accuracy: 0.8940 - val_loss: 0.2427
Epoch 6/80
301/301 ━━━━━━━━━━━━━━━━━━━━ 40s 133ms/step - accuracy: 0.8620 - loss: 0.3210 - val_accuracy: 0.9080 - val_loss: 0.2360
Epoch 7/80
301/301 ━━━━━━━━━━━━━━━━━━━━ 40s 133ms/step - accuracy: 0.8741 - loss: 0.3023 - val_accuracy: 0.8970 - val_loss: 0.2368
Epoch 8/80
301/301 ━━━━━━━━━━━━━━━━━━━━ 40s 132ms/step - accuracy: 0.8725 - loss: 0

In [14]:
loss,acc=model.evaluate(test_ds)
print()
print()
print("Accuracy: ",acc)
print()
print("Loss: ",loss)

32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 38ms/step - accuracy: 0.9260 - loss: 0.1870


Accuracy:  0.9240000247955322

Loss:  0.18862256407737732


In [15]:
import tensorflow as tf

# ✅ Convert the trained model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# ✅ Save the converted model
with open("skin_cancer.tflite", "wb") as f:
    f.write(tflite_model)

print("✅ TFLite model saved as 'skin_cancer.tflite'")


Saved artifact at '/tmp/tmp7axpeh0d'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 150, 150, 3), dtype=tf.float32, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  137446644952016: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137446644952784: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137446644952400: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137446644950672: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137446644952976: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137446644950480: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137446644951056: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137446644953360: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137446644954896: TensorSpec(shape=(), dtype=tf.resource, name=None)
  137446644953552: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13744664495547

# **Now adding slightly more complex model**

In [18]:
#CReate Deep Learning Model
model_cnn=Sequential()
model_cnn.add(layers.Input(shape=(150, 150, 3)))
model_cnn.add(data_augmentation)
model_cnn.add(Rescaling(1./255))
model_cnn.add(Conv2D(256,(3,3),padding='same',activation='relu'))
model_cnn.add(MaxPooling2D((2,2)))
model_cnn.add(Conv2D(256,(3,3),padding='same',activation='relu'))
model_cnn.add(MaxPooling2D((2,2)))
model_cnn.add(Conv2D(256,(3,3),padding='same',activation='relu'))
model_cnn.add(MaxPooling2D((2,2)))
model_cnn.add(Conv2D(256,(3,3),padding='same',activation='relu'))
model_cnn.add(MaxPooling2D((2,2)))
model_cnn.add(Conv2D(256,(3,3),padding='same',activation='relu'))
model_cnn.add(MaxPooling2D((2,2)))
model_cnn.add(Flatten())
model_cnn.add(Dense(128,activation='relu'))
model_cnn.add(Dropout(0.3))
model_cnn.add(Dense(256,activation='relu'))
model_cnn.add(Dropout(0.3))
model_cnn.add(Dense(256,activation='relu'))
model_cnn.add(Dropout(0.3))
model_cnn.add(Dense(1,activation='sigmoid'))

In [19]:
model_cnn.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential (Sequential)         │ (None, 150, 150, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling_2 (Rescaling)         │ (None, 150, 150, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 150, 150, 256)  │         7,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 75, 75, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 75, 75, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 37, 37, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 37, 37, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_11 (MaxPooling2D) │ (None, 18, 18, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_12 (Conv2D)              │ (None, 18, 18, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_12 (MaxPooling2D) │ (None, 9, 9, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_13 (Conv2D)              │ (None, 9, 9, 256)      │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_13 (MaxPooling2D) │ (None, 4, 4, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_2 (Flatten)             │ (None, 4096)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 128)            │       524,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,990,977 (11.41 MB)

 Trainable params: 2,990,977 (11.41 MB)

 Non-trainable params: 0 (0.00 B)

In [20]:
model_cnn.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

from keras.callbacks import EarlyStopping
early_stopping_cnn = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [21]:
cnn1=model_cnn.fit(
    train_ds,
    validation_data=test_ds,
    epochs=80,
    callbacks=[early_stopping_cnn],
    verbose=1,
)

Epoch 1/80
301/301 ━━━━━━━━━━━━━━━━━━━━ 52s 147ms/step - accuracy: 0.5071 - loss: 0.6942 - val_accuracy: 0.5000 - val_loss: 0.6956
Epoch 2/80
301/301 ━━━━━━━━━━━━━━━━━━━━ 43s 142ms/step - accuracy: 0.5240 - loss: 0.6930 - val_accuracy: 0.5000 - val_loss: 0.6932
Epoch 3/80
301/301 ━━━━━━━━━━━━━━━━━━━━ 43s 142ms/step - accuracy: 0.5205 - loss: 0.6929 - val_accuracy: 0.5000 - val_loss: 0.6957
Epoch 4/80
301/301 ━━━━━━━━━━━━━━━━━━━━ 43s 142ms/step - accuracy: 0.5213 - loss: 0.6916 - val_accuracy: 0.5000 - val_loss: 0.6944
Epoch 5/80
301/301 ━━━━━━━━━━━━━━━━━━━━ 43s 142ms/step - accuracy: 0.5389 - loss: 0.6777 - val_accuracy: 0.6990 - val_loss: 0.5460
Epoch 6/80
301/301 ━━━━━━━━━━━━━━━━━━━━ 43s 142ms/step - accuracy: 0.6467 - loss: 0.5849 - val_accuracy: 0.8440 - val_loss: 0.3286
Epoch 7/80
301/301 ━━━━━━━━━━━━━━━━━━━━ 43s 142ms/step - accuracy: 0.8283 - loss: 0.4087 - val_accuracy: 0.8630 - val_loss: 0.2979
Epoch 8/80
301/301 ━━━━━━━━━━━━━━━━━━━━ 43s 141ms/step - accuracy: 0.8374 - loss: 0

In [22]:
model_cnn.evaluate(test_ds)

32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 43ms/step - accuracy: 0.9322 - loss: 0.1869


[0.20287884771823883, 0.9169999957084656]

# **RESNET**

In [5]:
#Add pretrained model ResNet50

from tensorflow.keras.applications import ResNet50
base_model_resnet=ResNet50(include_top=False,input_shape=(224,224,3),weights='imagenet')

In [6]:
#Creating train,val and test data
train_ds_resnet=keras.utils.image_dataset_from_directory(
    train_dir,
    image_size=(224,224),
    batch_size=32,
    shuffle=True,
    color_mode="rgb",
    label_mode="binary",
    labels="inferred"
)

test_ds_resnet=keras.utils.image_dataset_from_directory(
    test_dir,
    image_size=(224,224),
    batch_size=32,
    shuffle=False,
    color_mode="rgb",
    label_mode="binary",
    labels="inferred"
)

Found 9605 files belonging to 2 classes.
Found 1000 files belonging to 2 classes.


In [7]:
from tensorflow.data import AUTOTUNE
train_ds_resnet=train_ds_resnet.cache().shuffle(5000).prefetch(buffer_size=AUTOTUNE)
test_ds_resnet=test_ds_resnet.cache().prefetch(buffer_size=AUTOTUNE)

In [8]:
#perform Data Augmentation
data_aug_resnet=Sequential([
    RandomFlip("horizontal"),
    RandomRotation(0.2),
    RandomZoom(0.2)

])

In [9]:
#Freeze all layers
for layer in base_model_resnet.layers:
  layer.trainable = False


#unfreeze last 100 layers
for layer in base_model_resnet.layers[-50:]:
  layer.trainable=True



inp=layers.Input(shape=(224,224,3))

x=data_aug_resnet(inp)
x = base_model_resnet(x, training=False)

#add own fully connected layers

x=GlobalAveragePooling2D()(x)
x=Dense(256,activation="relu",kernel_initializer='he_normal')(x)
x=Dropout(0.4)(x)
x=Dense(256,activation="relu",kernel_initializer='he_normal')(x)
x=Dropout(0.4)(x)
output = Dense(1, activation="sigmoid")(x)



model_resnet=Model(inp,output)

In [10]:
model_resnet.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential (Sequential)         │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resnet50 (Functional)           │ (None, 7, 7, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,178,305 (92.23 MB)

 Trainable params: 17,541,121 (66.91 MB)

 Non-trainable params: 6,637,184 (25.32 MB)

In [11]:
from tensorflow.keras.optimizers import Adam

model_resnet.compile(optimizer=Adam(learning_rate=1e-5), loss='binary_crossentropy', metrics=['accuracy'])


from keras.callbacks import EarlyStopping
early_stopping_resnet = EarlyStopping(monitor='val_loss', patience=6,verbose=1, restore_best_weights=True)

In [12]:
#fit the model
history_resnet=model_resnet.fit(train_ds_resnet,epochs=80,validation_data=test_ds_resnet,verbose=1,callbacks=[early_stopping_resnet])


Epoch 1/80
301/301 ━━━━━━━━━━━━━━━━━━━━ 102s 218ms/step - accuracy: 0.6719 - loss: 0.6230 - val_accuracy: 0.8960 - val_loss: 0.2382
Epoch 2/80
301/301 ━━━━━━━━━━━━━━━━━━━━ 64s 212ms/step - accuracy: 0.8608 - loss: 0.3167 - val_accuracy: 0.9090 - val_loss: 0.2175
Epoch 3/80
301/301 ━━━━━━━━━━━━━━━━━━━━ 65s 216ms/step - accuracy: 0.8928 - loss: 0.2707 - val_accuracy: 0.9120 - val_loss: 0.2047
Epoch 4/80
301/301 ━━━━━━━━━━━━━━━━━━━━ 65s 217ms/step - accuracy: 0.9052 - loss: 0.2259 - val_accuracy: 0.9140 - val_loss: 0.2060
Epoch 5/80
301/301 ━━━━━━━━━━━━━━━━━━━━ 66s 219ms/step - accuracy: 0.9121 - loss: 0.2116 - val_accuracy: 0.9160 - val_loss: 0.2045
Epoch 6/80
301/301 ━━━━━━━━━━━━━━━━━━━━ 66s 220ms/step - accuracy: 0.9126 - loss: 0.2060 - val_accuracy: 0.9190 - val_loss: 0.1885
Epoch 7/80
301/301 ━━━━━━━━━━━━━━━━━━━━ 66s 220ms/step - accuracy: 0.9277 - loss: 0.1816 - val_accuracy: 0.9170 - val_loss: 0.1945
Epoch 8/80
301/301 ━━━━━━━━━━━━━━━━━━━━ 66s 220ms/step - accuracy: 0.9303 - loss: 

In [13]:
resnet_loss,resnet_acc=model_resnet.evaluate(test_ds_resnet)
print()
print()
print("Accuracy: ",resnet_acc)
print()
print("Loss: ",resnet_loss)

32/32 ━━━━━━━━━━━━━━━━━━━━ 4s 112ms/step - accuracy: 0.9279 - loss: 0.1671


Accuracy:  0.9190000295639038

Loss:  0.18848057091236115
